In [ ]:
from deap import base, creator, tools, algorithms
import random

# Define the problem
def fitness(individual):
    a, b, c, d = individual
    return a * b * c * d,

# Constraint
def feasible(individual):
    return sum(individual) == 100

# Repair infeasible solution
def repair(individual):
    n = 100 / sum(individual)
    for i in range(len(individual)):
        individual[i] *= n
        individual[i] = int(individual[i])
    d = 100 - sum(individual)
    individual[random.randint(0, len(individual)-1)] += d
    return individual

# Setup DEAP
creator.create("FitnessMax", base.Fitness, weights=(1.0,))
creator.create("Individual", list, fitness=creator.FitnessMax)

toolbox = base.Toolbox()
toolbox.register("attr_int", random.randint, 1, 100)
toolbox.register("individual", tools.initRepeat, creator.Individual, toolbox.attr_int, 4)
toolbox.register("population", tools.initRepeat, list, toolbox.individual)
toolbox.register("evaluate", fitness)
toolbox.register("mate", tools.cxOnePoint)
toolbox.register("mutate", tools.mutUniformInt, low=1, up=100, indpb=0.2)
toolbox.register("select", tools.selTournament, tournsize=3)

# Decorate evaluation function with feasibility check
toolbox.decorate("evaluate", tools.DeltaPenalty(feasible, 0, repair))

# Evolutionary Algorithm
def main():
    pop = toolbox.population(n=50)
    hof = tools.HallOfFame(1)
    stats = tools.Statistics(lambda ind: ind.fitness.values)
    stats.register("max", max)

    algorithms.eaSimple(pop, toolbox, cxpb=0.5, mutpb=0.2, ngen=50000, stats=stats, halloffame=hof)

    return hof[0]

# Run the GA
best_ind = main()
print("Best Individual:", best_ind, "Fitness:", best_ind.fitness.values[0])
